# Day 15

Today, you set out on the task of perfecting your milk-dunking cookie recipe. All you have to do is find the right balance of ingredients.

Your recipe leaves room for exactly 100 teaspoons of ingredients. You make a list of the remaining ingredients you could use to finish the recipe (your puzzle input) and their properties per teaspoon:

    capacity (how well it helps the cookie absorb milk)
    durability (how well it keeps the cookie intact when full of milk)
    flavor (how tasty it makes the cookie)
    texture (how it improves the feel of the cookie)
    calories (how many calories it adds to the cookie)

You can only measure ingredients in whole-teaspoon amounts accurately, and you have to be accurate so you can reproduce your results in the future. The total score of a cookie can be found by adding up each of the properties (negative totals become 0) and then multiplying together everything except calories.

For instance, suppose you have these two ingredients:

```text
Butterscotch: capacity -1, durability -2, flavor 6, texture 3, calories 8
Cinnamon: capacity 2, durability 3, flavor -2, texture -1, calories 3
```

Then, choosing to use 44 teaspoons of butterscotch and 56 teaspoons of cinnamon (because the amounts of each ingredient must add up to 100) would result in a cookie with the following properties:

    A capacity of 44*-1 + 56*2 = 68
    A durability of 44*-2 + 56*3 = 80
    A flavor of 44*6 + 56*-2 = 152
    A texture of 44*3 + 56*-1 = 76

Multiplying these together (68 * 80 * 152 * 76, ignoring calories for now) results in a total score of 62842880, which happens to be the best score possible given these ingredients. If any properties had produced a negative total, it would have instead become zero, causing the whole score to multiply to zero.

Given the ingredients in your kitchen and their properties, what is the total score of the highest-scoring cookie you can make?

## Puzzle 1

In [1]:
import random

from dataclasses import dataclass
from typing import Iterable

import numpy as np


@dataclass
class Ingredient():
    
    name: str
    capacity: int
    durability: int
    flavor: int
    texture: int
    calories: 8


def parse_ingredient(instr:str):
    """Return Ingredient object
    
    :param instr: string describing ingredient
    """
    data = instr.strip().replace(",", "").split()
    ing = Ingredient(data[0][:-1], int(data[2]), int(data[4]), int(data[6]), int(data[8]), int(data[10]))
    return ing


def calculate_score(ingredients:Iterable, quantities:Iterable):
    """Return score of ingredients
    
    :param ingredients: iterable of Ingredient objects
    :param quantities: iterable of quantities of each ingredient
    """
    cap, dur, fla, tex, cal = 0, 0, 0, 0, 0
    
    for i, q in zip(ingredients, quantities):
        cap += i.capacity * q
        dur += i.durability * q
        fla += i.flavor * q
        tex += i.texture * q
    
#     print(quantities, [_ for _ in (cap, dur, fla, tex)])
    return np.prod([max(0, _) for _ in (cap, dur, fla, tex)])
    

def find_best_neighbour(ingredients, quantities):
    neighbours = []
    for idx1, qty in enumerate(quantities[:-1]):
        for idx2 in range(idx1+1, len(quantities)):
            for val in (1, -1):
                nbr = quantities[:]
                nbr[idx1] += val
                nbr[idx2] += val * -1
                if (0 < nbr[idx1] < 100) and (0 < nbr[idx2] < 100):
                    neighbours.append((calculate_score(ingredients, nbr), nbr))
#     print(neighbours)
    return sorted(neighbours, reverse=True)[0]
    
    
def optimise(ingredients:Iterable, total=100, quantities=None):
    """Optimise quantities of passed ingredients
    
    :param ingredients: iterable of Ingredient objects
    :param total: total amount of ingredients
    """
    if quantities == None:
        quantities = [total/len(ingredients)] * len(ingredients)
    cost = calculate_score(ingredients, quantities)  # initial cost

    # Find best immediate neighbour (+/- 1 unit in a pair of ingredients)
    nbrscore, nbr = find_best_neighbour(ingredients, quantities)
    while nbrscore > cost:
        cost, quantities = nbrscore, nbr
        nbrscore, nbr = find_best_neighbour(ingredients, quantities)
#         print(nbrscore, nbr)
    print(int(cost), list(zip(quantities, [ing.name for ing in ingredients])))

In [2]:
ingredients = ("Butterscotch: capacity -1, durability -2, flavor 6, texture 3, calories 8",
               "Cinnamon: capacity 2, durability 3, flavor -2, texture -1, calories 3")

for ing in ingredients:
    print(ing, parse_ingredient(ing))
    
optimise([parse_ingredient(ing) for ing in ingredients])

Butterscotch: capacity -1, durability -2, flavor 6, texture 3, calories 8 Ingredient(name='Butterscotch', capacity=-1, durability=-2, flavor=6, texture=3, calories=8)
Cinnamon: capacity 2, durability 3, flavor -2, texture -1, calories 3 Ingredient(name='Cinnamon', capacity=2, durability=3, flavor=-2, texture=-1, calories=3)
62842880 [(44.0, 'Butterscotch'), (56.0, 'Cinnamon')]


### Solution

In [3]:
with open("day15.txt", "r") as ifh:
    optimise([parse_ingredient(ing) for ing in ifh.readlines()],
             quantities = [24, 15, 25, 36])

222870 [(21, 'Sugar'), (5, 'Sprinkles'), (31, 'Candy'), (43, 'Chocolate')]


## Puzzle 2

Your cookie recipe becomes wildly popular! Someone asks if you can make another recipe that has exactly 500 calories per cookie (so they can use it as a meal replacement). Keep the rest of your award-winning process the same (100 teaspoons, same ingredients, same scoring system).

For example, given the ingredients above, if you had instead selected 40 teaspoons of butterscotch and 60 teaspoons of cinnamon (which still adds to 100), the total calorie count would be 40*8 + 60*3 = 500. The total score would go down, though: only 57600000, the best you can do in such trying circumstances.

Given the ingredients in your kitchen and their properties, what is the total score of the highest-scoring cookie you can make with a calorie total of 500?

### Solution

In [4]:
from itertools import permutations

# Brute force generation of all valid quantities
candidates = [_ for _  in permutations(list(range(1, 101)), 4) if sum(_) == 100]

In [5]:
def total_calories(ingredients, quantities):
    return sum([i.calories * q for (i, q) in zip(ingredients, quantities)])

### Solution

In [6]:
with open("day15.txt", "r") as ifh:
    ingredients = [parse_ingredient(ing) for ing in ifh.readlines()]  # read ingredients
    candidates2 = [_ for _ in candidates if total_calories(ingredients, _) == 500]  # all recipes with 500 calories
    scores = [(calculate_score(ingredients, _), _) for _ in candidates2]  # all scores for 500 calorie recipes

sorted(scores, reverse=True)[0]  # Highest-scoring 500 calorie recipe

(117936, (21, 8, 26, 45))